In [435]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # enables 3D projection

def compute_pairwise_distances(
    coords1: np.ndarray, # shape (N1, 2)
    coords2: np.ndarray  # shape (N2, 2)
) -> np.ndarray:  # result shape (N1, N2)

    diffs = coords1[:, None, :] - coords2[None, :, :] # shape (N1, N2, 2)
    return np.linalg.norm(diffs, axis=2)              # shape (N1, N2)

def compute_D_matrix(
    coords1: np.ndarray, # shape (N1, 2)
    coords2: np.ndarray, # shape (N2, 2)
    baseline: np.ndarray # shape (2,)
) -> np.ndarray:  # result shape (N1, N2)

    d1 = np.linalg.norm(coords1 - baseline, axis=1)    # shape (N1,)
    d2 = np.linalg.norm(coords2 - baseline, axis=1)    # shape (N2,)
    d12 = compute_pairwise_distances(coords1, coords2) # shape (N1, N2)
    return (d1[:, None] + d2[None, :] - d12) / 2       # shape (N1, N2)

def compute_D_inv_ext(
    coords1: np.ndarray, # shape (N1, 2)
    coords2: np.ndarray, # shape (N2, 2)
    baseline: np.ndarray # shape (2,)
) -> np.ndarray:  # result shape (N1, N2)
    D = compute_D_matrix(coords1, coords2, baseline)
    D_inv = np.linalg.inv(D)
    D_inv1 = -D_inv @ np.ones((D_inv.shape[0], 1))
    D_inv11 = -np.ones(D_inv.shape[0]) @ D_inv1[:,0]
    D_inv_ext = np.block([[D_inv11, D_inv1.T],[D_inv1, D_inv]])
    return D_inv_ext

import numpy as np

def extend_random_field(
    existing_coords: np.ndarray,    # shape (Ne, 2)
    existing_phi: np.ndarray,       # shape (Ne,)
    new_coords: np.ndarray,         # shape (Nn, 2)
    nu: float                       # scalar
) -> tuple[np.ndarray, np.ndarray]: # all_coords shape (Ne+Nn, 2), all_phi shape (Ne+Nn,)

    baseline = existing_coords[0]   # shape (2,)
    phi0 = existing_phi[0]          # scalar
    coords_e = existing_coords[1:]  # shape (Ne-1, 2)
    phi_e = existing_phi[1:] - phi0 # shape (Ne-1,)

    D_ee = compute_D_matrix(coords_e, coords_e, baseline)     # shape (Ne-1, Ne-1)
    D_en = compute_D_matrix(coords_e, new_coords, baseline)   # shape (Ne-1, Nn)
    D_ne = D_en.T                                             # shape (Nn, Ne-1)
    D_nn = compute_D_matrix(new_coords, new_coords, baseline) # shape (Nn, Nn)

    inv_Dee = np.linalg.inv(D_ee)                             # shape (Ne-1, Ne-1)
    mu_n = phi0 + D_ne @ inv_Dee @ phi_e                      # shape (Nn,)
    Dcov_n = D_nn - D_ne @ inv_Dee @ D_en                     # shape (Nn, Nn)
    phi_n = np.random.multivariate_normal(mu_n, nu * Dcov_n)  # shape (Nn,)

    all_coords = np.vstack([existing_coords, new_coords])     # (Ne+Nn, 2)
    all_phi = np.concatenate([existing_phi, phi_n])           # (Ne+Nn,)
    return all_coords, all_phi


In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

size = 50

coords1 = np.stack(np.mgrid[0:size, 0:size], axis=-1).reshape(-1, 2)[1:]
D_inv_ext = compute_D_inv_ext(coords1, coords1, np.array([0.0, 0.0]))

A = D_inv_ext     # shape (100, 100) if you’re using a 10×10 grid

# integer sliders 0–9
sx = widgets.IntSlider(min=0, max=size - 1, value=0, description='x1')
sy = widgets.IntSlider(min=0, max=size - 1, value=0, description='y1')
tx = widgets.IntSlider(min=0, max=size - 1, value=0, description='x2')
ty = widgets.IntSlider(min=0, max=size - 1, value=0, description='y2')

# must be an ipywidgets widget, not IPython.display.HTML
out = widgets.HTML()

def update(change=None):
    i = sx.value * size + sy.value
    j = tx.value * size + ty.value
    v1 = A[i, j]
    out.value = f"<b>A[{i}, {j}] = {v1:.6f}</b>"

for w in (sx, sy, tx, ty):
    w.observe(update, names='value')

update()
display(widgets.VBox([sx, sy, tx, ty, out]))

In [ ]:
sizeline = 4
coordsline = np.column_stack((np.arange(sizeline), np.zeros(sizeline, dtype=int)))[1:]
D_inv_extline = compute_D_inv_ext(coordsline, coordsline, np.array([0.0, 0.0]))
D_inv_extline

array([[ 1., -1.,  0.,  0.],
       [-1.,  2., -1.,  0.],
       [ 0., -1.,  2., -1.],
       [ 0.,  0., -1.,  1.]])

$D^{-1}$

In [ ]:
8 * np.linalg.pinv(D_inv_extline)

array([[ 7.,  1., -3., -5.],
       [ 1.,  3., -1., -3.],
       [-3., -1.,  3.,  1.],
       [-5., -3.,  1.,  7.]])

In [ ]:
D_matrix = compute_D_matrix(coordsline, coordsline, np.array([0.0, 0.0]))
ext3 = get_ext(3)
np.linalg.pinv(ext3 @ np.linalg.inv(D_matrix) @ ext3.T)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)

In [ ]:
D_matrix

array([[1., 1., 1.],
       [1., 2., 2.],
       [1., 2., 3.]])

In [ ]:
allcoordsline = np.column_stack((np.arange(sizeline), np.zeros(sizeline, dtype=int)))
to_D = get_ext(3)
- 1/2 * to_D.T @ compute_pairwise_distances(allcoordsline, allcoordsline) @ to_D

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)

In [ ]:
ext3_pinv = np.linalg.pinv(ext3)
ext3_pinv

array([[-0.25,  0.75, -0.25, -0.25],
       [-0.25, -0.25,  0.75, -0.25],
       [-0.25, -0.25, -0.25,  0.75]])

In [ ]:
def get_ext(n):
    return np.block([[-np.ones(n-1)],[np.eye(n-1)]])
def get_ext_pinv(n):
    return np.block([np.zeros((n-1,1)),np.eye(n-1)]) - np.ones((n-1,n))/n
for n in range(1,20):
    assert np.allclose(np.linalg.pinv(get_ext(n)), get_ext_pinv(n))

# This one has the cool property that get_ext_combo(n) = get_ext_combo(n).T = np.linalg.pinv(get_ext_combo(n))
# What it effectively does is centers the input vector by subtracting the mean from all elements.
def get_centerer(n):
    return np.eye(n)-np.ones((n,n))/n

In [ ]:
D_ext = - 1/2 * get_centerer(4) @ compute_pairwise_distances(allcoordsline, allcoordsline) @ get_centerer(4)

NameError: name 'allcoordsline' is not defined

In [ ]:
def get_centerer(n):
    return np.eye(n)-np.ones((n,n))/n
D_inv_ext2 = np.linalg.pinv(D_ext)

NameError: name 'D_ext' is not defined

In [ ]:
phi = np.array([
    1.,
    0,
    0,
    0,
])
phi -= np.mean(phi)
phi

array([ 0.75, -0.25, -0.25, -0.25])

In [ ]:
D_inv_ext2 @ phi

array([ 1.00000000e+00, -1.00000000e+00, -5.55111512e-17,  1.94289029e-16])

In [ ]:
phi2 = np.linalg.solve(D_ext, phi)
phi2 -= np.mean(phi2)
phi2 # We hope that this has nice properties or something

array([ 1., -1.,  0.,  0.])

In [ ]:
D_ext @ phi2 + np.ones(4) / 4 # Given vertex V, we hope that if phi2 only has support around the vertices adjacent to V, then the output here is mostly at V

array([ 1.00000000e+00,  1.11022302e-16,  0.00000000e+00, -1.11022302e-16])

In [ ]:
-2 * get_centerer(4) @ np.linalg.pinv(compute_pairwise_distances(allcoordsline, allcoordsline)) @ get_centerer(4)

array([[ 0.91666667, -0.91666667,  0.08333333, -0.08333333],
       [-0.91666667,  1.91666667, -1.08333333,  0.08333333],
       [ 0.08333333, -1.08333333,  1.91666667, -0.91666667],
       [-0.08333333,  0.08333333, -0.91666667,  0.91666667]])

In [ ]:
D_inv_extline

array([[ 1., -1.,  0.,  0.],
       [-1.,  2., -1.,  0.],
       [ 0., -1.,  2., -1.],
       [ 0.,  0., -1.,  1.]])

In [ ]:
phi2(i,j<=i) free
D_ext[I,J] = dist(point(I), point(J))


In [ ]:
D_inv @ e(k) = phi
i1 >= 0, i2 >= 0
sum_(j1 in {-inf,...,0,...,inf}) sum_j2 sqrt((i1 - j1)**2 + (i2 - j2)**2) * phi(j1, j2) = {(0, 0)=(i1, i2)}

D_inv = np.zeros
D_inv[0,0,0,0] = alpha
D_inv[0,0,1,0] = beta
D_inv[0,0,0,1] = beta
D_inv[0,0,1,1] = gamma
D_inv[0,1,0,1] = alpha
D_inv[0,1,0,2] = beta
D_inv[0,1,1,1] = beta
D_inv[]

In [ ]:
phi(0,0) = 0
pdf(0,1) = phi(0,0) + N(mu)
pdf(1,0) = phi(0,0) + N(mu)
pdf(1,1) = p * phi(0,1) + p * phi(1,0) + (1 - 2*p) * phi(0,0) + N(nu)

In [100]:
import sympy as sp
import sympy.stats as ss

In [94]:
def cov(phis, N):
    return sp.Matrix([[ss.covariance(phis[i], phis[j]) if i != j else ss.variance(phis[i]) for j in range(N)] for i in range(N)])

def get_extention_cov(
    existing_coords: np.ndarray,    # shape (Ne, 2)
    new_coords: np.ndarray,         # shape (Nn, 2)
):

    baseline = existing_coords[0]   # shape (2,)
    coords_e = existing_coords[1:]  # shape (Ne-1, 2)
    D_ee = compute_D_matrix(coords_e, coords_e, baseline)     # shape (Ne-1, Ne-1)
    D_en = compute_D_matrix(coords_e, new_coords, baseline)   # shape (Ne-1, Nn)
    D_ne = D_en.T                                             # shape (Nn, Ne-1)
    D_nn = compute_D_matrix(new_coords, new_coords, baseline) # shape (Nn, Nn)

    inv_Dee = np.linalg.inv(D_ee)                             # shape (Ne-1, Ne-1)
    Dcov_n = D_nn - D_ne @ inv_Dee @ D_en                     # shape (Nn, Nn)

    return Dcov_n

In [178]:
COV_full = get_extention_cov(np.array([[0,0]]), np.array([[0,2], [2,0], [2,2]]))
print(COV_full)

[[2.     0.5858 1.4142]
 [0.5858 2.     1.4142]
 [1.4142 1.4142 2.8284]]


In [517]:
def e(i, j, n):
    arr = np.zeros(n ** 2)
    arr[I(i, j, n)] = 1.0
    return arr

def I(i, j, n):
    return n * i + j

kappa, mu, nu, p, q = sp.symbols('kappa mu nu p q')

def get_phis3(kappa, mu, nu, p, q):
    phis = np.zeros((9, 9))
    phis[I(0, 0, 3), :] = np.zeros(9) # Redundant
    phi00 = np.zeros(9)
    phis[I(0, 1, 3), :] = phis[I(0, 0, 3), :] + kappa * e(0, 1, 3)
    phis[I(1, 0, 3), :] = p * phis[I(0, 0, 3), :] + (1 - p) * phis[I(0, 1, 3), :] + mu * e(1, 0, 3)
    phis[I(1, 1, 3), :] = q * phis[I(0, 1, 3), :] + q * phis[I(1, 0, 3), :] + (1 - 2 * q) * phis[I(0, 0, 3), :] + nu * e(1, 1, 3)
    phis[I(0, 2, 3), :] = p * phis[I(0, 1, 3), :] + (1 - p) * phis[I(1, 1, 3), :] + mu * e(0, 2, 3)
    phis[I(1, 2, 3), :] = q * phis[I(1, 1, 3), :] + q * phis[I(0, 2, 3), :] + (1 - 2 * q) * phis[I(0, 1, 3), :] + nu * e(1, 2, 3)
    phis[I(2, 0, 3), :] = p * phis[I(1, 0, 3), :] + (1 - p) * phis[I(1, 1, 3), :] + mu * e(2, 0, 3)
    phis[I(2, 1, 3), :] = q * phis[I(1, 1, 3), :] + q * phis[I(2, 0, 3), :] + (1 - 2 * q) * phis[I(1, 0, 3), :] + nu * e(2, 1, 3)
    phis[I(2, 2, 3), :] = q * phis[I(2, 1, 3), :] + q * phis[I(1, 2, 3), :] + (1 - 2 * q) * phis[I(1, 1, 3), :] + nu * e(2, 2, 3)
    return phis

def get_phis(n, kappa, nu, p, q):
    mu = kappa * np.sqrt(p * (2 - p))
    phis = np.zeros((n ** 2, n ** 2))
    for m in range(1, n):
        if m == 1:
            phis[I(0, m, n), :] = kappa * e(0, m, n)
        else:
            phis[I(0, m, n), :] = p * phis[I(0, m - 1, n), :] + (1 - p) * phis[I(1, m - 1, n), :] + mu * e(0, m, n)
            for i in range(1, m): # Terminate at i = m - 1
                phis[I(i, m, n), :] = q * phis[I(i, m - 1, n), :] + q * phis[I(i - 1, m, n), :] + (1 - 2 * q) * phis[I(i - 1, m - 1, n), :] + nu * e(i, m, n)
        phis[I(m, 0, n), :] = p * phis[I(m - 1, 0, n), :] + (1 - p) * phis[I(m - 1, 1, n), :] + mu * e(m, 0, n)
        for i in range(1, m + 1): # Terminate at i = m
            phis[I(m, i, n), :] = q * phis[I(m - 1, i, n), :] + q * phis[I(m, i - 1, n), :] + (1 - 2 * q) * phis[I(m - 1, i - 1, n), :] + nu * e(m, i, n)
    return phis

def get_phicov(n, *args):
    phis = get_phis(n, *args)
    return phis @ phis.T / n

def select(phicov, n, indices):
    iindices = list(map(lambda tup: I(tup[0], tup[1], n), indices))
    return phicov[iindices, :][:, iindices]

def get_cov_corners(n, *args):
    phicov = get_phicov(n, *args)
    indices = [(0, n - 1), (n - 1, 0), (n - 1, n - 1)]
    return select(phicov, n, indices)

# Will print if there is an asymmetry instead of raising an exception
def check_cov_symm(phicov, n):
    for i1 in range(n):
        for j1 in range(n):
            for i2 in range(n):
                for j2 in range(n):
                    if not np.isclose(phicov[I(j1, i1, n), I(j2, i2, n)], phicov[I(i1, j1, n), I(i2, j2, n)]):
                        print(f"inconsistency at {i1=} {j1=} {i2=} {j2=}")

np.set_printoptions(linewidth=90, precision=4)

COV = get_extention_cov(np.array([[0,0]]), np.array([[0,1], [1,0], [1,1]]))
def phicov_pq(n, p, q):
    kappaonly = get_cov_corners(n, 1, 0, p, q)
    # print(f"{kappaonly.shape=}")
    nuonly = get_cov_corners(n, 0, 1, p, q)
    # print(f"{nuonly.shape=}")
    # kkappa * kappaonly + knu * nuonly = COV
    # kkappa * kappaonly[0,0] + knu * nuonly[0,0] = COV[0,0]
    # kkappa * kappaonly[0,1] + knu * nuonly[0,1] = COV[0,1]
    mat = np.array([
        [kappaonly[0,0], nuonly[0,0]],
        [kappaonly[-1,-1], nuonly[-1,-1]]
    ])
    # print(mat)
    ks, _, _, _ = np.linalg.lstsq(mat, np.array([COV[0,0], COV[-1,-1]]))
    phicov = ks[0] * kappaonly + ks[1] * nuonly
    return phicov

def get_loss(n, p, q):
    return np.linalg.matrix_norm(phicov_pq(n, p, q) - COV)
def autograd(n, p, q, eps=1e-6):
    loss0 = get_loss(n, p, q)
    lossp = get_loss(n, p + eps, q)
    lossq = get_loss(n, p, q + eps)
    return (loss0, np.array([(lossp - loss0) / eps, (lossq - loss0) / eps]))

In [509]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

n = 100

# Create grid of p and q values
p_values = np.arange(0, 1, 0.1)
q_values = np.arange(0, 0.5, 0.05)

# Create meshgrid
P, Q = np.meshgrid(p_values, q_values)

# Compute loss for each combination
Z = np.zeros_like(P)
for i, p in enumerate(p_values):
    for j, q in enumerate(q_values):
        Z[j, i] = np.log(get_loss(n, p, q))

# Create 3D plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(P, Q, Z, cmap='viridis', edgecolor='none', alpha=0.8)
ax.set_xlabel('p')
ax.set_ylabel('q')
ax.set_zlabel('Loss')
ax.set_title(f'Loss(n={n}, p, q) - 3D Surface')

plt.colorbar(surf, ax=ax, shrink=0.5)
plt.show()

KeyboardInterrupt: 